In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.gaussian_process as gp
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

import bayesian optimization module by thomas huijskens

In [19]:
from google.colab import files
src = list(files.upload().values())[0]
open('gp.py','wb').write(src)
import gp

%load gp.py
%run gp.py

Saving gp.py to gp (1).py


import transformer dataset

In [20]:
from google.colab import files
uploaded = files.upload()
import io
data = pd.read_csv(io.BytesIO(uploaded['Overview.csv']))

Saving Overview.csv to Overview (3).csv


In [21]:
data.shape

(20316, 8)

In [22]:
data.head(n=7)

,DeviceTimeStamp,OTI,WTI,ATI,OLI,OTI_A,OTI_T,MOG_A
0,2019-06-25T13:06,0,0,0,37,0,0,0
1,2019-06-25T13:09,0,0,0,37,0,0,0
2,2019-06-27T10:49,0,0,0,38,0,0,0
3,2019-06-27T10:51,0,0,0,38,0,0,0
4,2019-06-27T10:52,0,0,0,39,0,0,0
5,2019-06-27T10:52,0,0,0,38,0,0,0
6,2019-06-27T10:53,0,0,0,39,0,0,0


feature engineering

In [23]:
nms = list(data.columns.values)
p = len(nms)

x_nm = ['OTI','WTI','ATI','OLI','OTI_A','MOG_A']
y_nm = ['OTI_T']

print('Feature variables are:', x_nm)
print('Target variables are:', y_nm)

X = data[x_nm]
y = data[y_nm]

Feature variables are: ['OTI', 'WTI', 'ATI', 'OLI', 'OTI_A', 'MOG_A']
Target variables are: ['OTI_T']


In [24]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

SVM classification with bayesian optimization

In [25]:
## define loss function

def sample_loss(params):
  C = params[0]
  gamma = params[1]

  # sample C and gamma on the log-uniform scale
  model = SVC( C = 10**C, gamma = 10**gamma, random_state = 23 )

  # sample parameters on a log scale
  return cross_val_score(model, X=X_train, y=y_train, scoring='accuracy', cv=3).mean()

Define the boundry and run the bayesian optimizationon on SVM

In [26]:
bounds = np.array([[-10, 10], [-10, 10]])
print(bounds)

np.random.seed(seed=10)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss,
                               bounds= bounds,
                               n_pre_samples=10)

# the maximum is at:
xp_hat = xp [np.array(yp).argmax(), :]

print(xp_hat)

[[-10  10]
 [-10  10]]
[ 5.42641287 -9.58496101]


optimal choice of C and gamma we are fitting the model

In [27]:
print(10**xp_hat)

SVC_best = SVC(C=10**xp_hat[0], gamma=10**xp_hat[1])

SVC_best.fit(X_train, y_train)

print('In-sample score =', SVC_best.score(X_train, y_train)*100)
print('Out-sample score =',SVC_best.score(X_test, y_test)*100)

[2.66939514e+05 2.60039299e-10]
In-sample score = 100.0
Out-sample score = 100.0
